## Convolutional Neural Networks

### Load train file into a Panda DataFrame

In [2]:
import numpy as np
import pandas as pd

input_file = "./../Sample/train.csv"
mnist = pd.read_csv(input_file, header = 0)

pd.DataFrame.head(mnist, 10)

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,7,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,5,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### Import TensorFlow and set up grid size and other variables

In [3]:
import tensorflow as tf

height = 28
width = 28
channels = 1
n_inputs = height * width

conv1_fmaps = 32
conv1_ksize = 3
conv1_stride = 1
conv1_pad = "SAME"

conv2_fmaps = 64
conv2_ksize = 3
conv2_stride = 2
conv2_pad = "SAME"

pool3_fmaps = conv2_fmaps

n_fc1 = 64
n_outputs = 10

### Reset the graph per run.  Used in case changes are made.

In [4]:
# to make this notebook's output stable across runs
def reset_graph(seed=42):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)

### Create the pooling layers

In [5]:
reset_graph()

with tf.name_scope("inputs"):
    X = tf.placeholder(tf.float32, shape=[None, n_inputs], name="X")
    X_reshaped = tf.reshape(X, shape=[-1, height, width, channels])
    y = tf.placeholder(tf.int32, shape=[None], name="y")

conv1 = tf.layers.conv2d(X_reshaped, filters=conv1_fmaps, kernel_size=conv1_ksize,
                         strides=conv1_stride, padding=conv1_pad,
                         activation=tf.nn.relu, name="conv1")

maxpool1 = tf.layers.max_pooling2d(conv1, pool_size=[2, 2], strides=2)

conv2 = tf.layers.conv2d(maxpool1, filters=conv2_fmaps, kernel_size=conv2_ksize,
                         strides=conv2_stride, padding=conv2_pad,
                         activation=tf.nn.relu, name="conv2")

with tf.name_scope("conv2_flat"):
    conv2_flat = tf.reshape(conv2, shape=[-1, pool3_fmaps * 7 * 7])
    
with tf.name_scope("fc1"):
    fc1 = tf.layers.dense(conv2_flat, n_fc1, activation=tf.nn.relu, name="fc1")
    
with tf.name_scope("output"):
    logits = tf.layers.dense(fc1, n_outputs, name="output")
    Y_proba = tf.nn.softmax(logits, name="Y_proba")

### Calculates the loss and the cross entropy and sets the learning rate and the training optimizer

In [6]:
with tf.name_scope("train"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits, labels=y)
    loss = tf.reduce_mean(xentropy)
    optimizer = tf.train.AdamOptimizer()
    training_op = optimizer.minimize(loss)

### Evaluates the model as it learns and evaluates it's accuracy

In [7]:
with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

### Set the initializer and saver for the training session
- Saver stores the trained neural network

In [8]:
with tf.name_scope("init_and_save"):
    init = tf.global_variables_initializer()
    saver = tf.train.Saver()

### Train neural network and print out it's accuracy against the training data

In [9]:
n_epochs = 10
batch_size = 100
num_examples = len(mnist)

with tf.Session() as sess:
    init.run()
    for epoch in range(n_epochs):
        for iteration in range(num_examples // batch_size):
            start, end  = iteration * batch_size, (iteration + 1) * batch_size
            X_batch, y_batch = mnist.iloc[start:end, 1:] / 255, mnist.iloc[start:end, 0]
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
        acc_train = accuracy.eval(feed_dict={X: X_batch, y: y_batch})
        print(epoch, "Train accuracy:", acc_train)

        save_path = saver.save(sess, "./my_mnist_model")

0 Train accuracy: 0.99
1 Train accuracy: 0.99
2 Train accuracy: 1.0
3 Train accuracy: 1.0
4 Train accuracy: 1.0
5 Train accuracy: 1.0
6 Train accuracy: 1.0
7 Train accuracy: 1.0
8 Train accuracy: 1.0
9 Train accuracy: 1.0


### Load test file into a Panda DataFrame

In [10]:
test_input_file = "./../Sample/test.csv"
test_mnist = pd.read_csv(test_input_file, header = 0)

pd.DataFrame.head(test_mnist, 10)

,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### Predict the the values for the test set and save the results

In [11]:
with tf.Session() as sess:
    saver.restore(sess, "./my_mnist_model") # or better, use save_path
    X_new_scaled = test_mnist
    Z = logits.eval(feed_dict={X: X_new_scaled})
    y_pred = np.argmax(Z, axis=1)

INFO:tensorflow:Restoring parameters from ./my_mnist_model


### Print predicted results

In [12]:
print("Predicted classes:", y_pred)

Predicted classes: [2 0 9 ... 3 9 2]


### Export results to CSV to be uploaded to Kaggle

In [13]:
row = range(1, y_pred.size + 1)
result_df = pd.DataFrame({
    'ImageId': row,
    'Label': y_pred
})

result_df[['ImageId', 'Label']].to_csv('./convolutional_neural_networks_results.csv', sep=',', index = False, encoding='utf-8')